In [1]:
# 原始图像地址
import numpy as np
import cv2
from matplotlib import pyplot as plt
# 图片显示
def show_plt(imgs_tuple):
    """ 图像显示。传参：图像元组 返回值：无 """
    out_imgs=np.hstack(imgs_tuple)
    out_imgs = cv2.cvtColor(out_imgs,cv2.COLOR_RGBA2BGR)
    plt.imshow(out_imgs)

# K-Means聚类--用于圆形拟合视杯区域
def kmeans(roi):
    # K-Means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    flags = cv2.KMEANS_RANDOM_CENTERS
    X = roi.reshape((-1, 1))
    X = np.float32(X)
    _,labels,centers = cv2.kmeans(X,2,None,criteria,2,flags)

    centers[labels[0]]=[0]
    centers = np.uint8(centers)
    res= centers[labels.flatten()]
    res2 = res.reshape((roi.shape))
    return res2
# 霍夫曼圆形拟合
def HoughCircles_trans(img):
    HoughCircles_img=img.copy()
    circle1 = cv2.HoughCircles(HoughCircles_img, cv2.HOUGH_GRADIENT, 1, 3000, param1=200, param2=0.9, minRadius=100, maxRadius=256)  #把半径范围缩小点，检测内圆，瞳孔
    # print(circle1.shape)
    circles = circle1[0, :, :]  # 提取为二维
    circles = np.uint16(np.around(circles))  # 四舍五入，取整
    # show_plt((circles,))
    return circles[0]

# 膨胀
def dilate_img(thresh3):
    # kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    # dst = cv2.dilate(thresh3, kernel1)
    return thresh3

# 选择通道
def channel_selct(img,channel="r"):
    b,g,r = cv2.split(img)
    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    g_=r
    if channel=="b":
        g_=b
    elif(channel=="g"):
        g_=g
    elif(channel=="gray"):
        g_=img_gray
    return g_

def cv_function_r(img_path,channel="r",file_name="test.png"):
    img = cv2.imread(img_path)
    # 选择通道
    img_c=channel_selct(img,channel="r")
    g_=img_c.copy()
    # # 直方图
    # g_=cv2.equalizeHist(g_)
    # eq_img=g_.copy()
    # 对直方图处理的图片进行聚类分析，得到视杯区域
    g_=kmeans(g_)
    kmeans_img=g_.copy()
    # 膨胀处理，消除血管位置对视盘区域识别的影响
    dilate_img_=dilate_img(g_.copy())
    dil_img_=dilate_img_.copy()
    # 霍夫曼圆形拟合，得到【x,y,半径】
    info_circle=HoughCircles_trans(dilate_img_)
    # print(f"info_circle{info_circle}")
    # 得到mask
    xxx__=cv2.circle(img_c.copy(), (info_circle[0], info_circle[1]), info_circle[2]+50, 255, -1)  # 画圆
    _,mask__=cv2.threshold(xxx__,254,255,cv2.THRESH_BINARY)
    img_hough=img_c.copy()
    img_hough = cv2.add(img_hough, np.zeros(np.shape(img_hough), dtype=np.uint8), mask=mask__)
    to_save_img=np.hstack((img_c,kmeans_img,dil_img_,mask__,img_hough))
    cv2.imwrite(f"/home/wzc/zlt_self/CV_Work/hough_aft/{file_name}",to_save_img)


In [2]:
img_path="/home/wzc/zlt_self/pOSAL-master/data/refuge/train0/disc_small/image/n0112.png"

cv_function_r(img_path)

In [3]:
import os
# 判断存储位置是否存在，没有则创建
save_hough_path="/home/wzc/zlt_self/CV_Work/hough_aft"
if not os.path.exists(save_hough_path):
    os.mkdir(save_hough_path)
# 获得所有img地址
# root_path=f"/home/wzc/zlt_self/ZNewMyModel/data/Drishti-GS/test/disc_small/image"
sub_dir=["Drishti-GS","refuge"]
next_dir=["train","test"]
for sub_dir_ in sub_dir:
    for next_dir_ in next_dir:
        img_dir=f"/home/wzc/zlt_self/ZNewMyModel/data/{sub_dir_}/{next_dir_}/disc_small/image"
        if os.path.exists(img_dir):
            file_names=os.listdir(img_dir)
            print(f"length:{file_names.__len__()}---{sub_dir_}/{next_dir_}/disc_small/image")
            for file_name in file_names:
                img_path=os.path.join(img_dir,file_name)
                cv_function_r(img_path,file_name=file_name,channel="gray")
        else:
                print(f"！！！不存在：{img_dir}！！！")

length:50---Drishti-GS/train/disc_small/image
length:51---Drishti-GS/test/disc_small/image
length:321---refuge/train/disc_small/image
！！！不存在：/home/wzc/zlt_self/ZNewMyModel/data/refuge/test/disc_small/image！！！


In [4]:
[1,2,3].__len__()

3